In this file we will be combining the chess figure detection and the chessboard detection.
We will take the output of the machine learning model and use it to calculate the position on the chessboard.

In [2]:
import glob,torch
import matplotlib.pyplot as plt
from chessLocalisation import *
from figureAssignment import *
from chessboard import *
from helpfulUtils import *

#Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='weights/best.pt')  # local model
model.conf = 0.25

folder = ['board1_set1','board1_set2','board2_set1','board2_set2','light_board1_set1','light_board1_set2','light_board2_set1','light_board2_set2']
for f in folder:
    filepath = '../datasets/Chess-Pieces_check_Accuracy/' + f + '/'
    setup_image = filepath + 'setup.JPG'
    chess_images = glob.glob(filepath + 'IMG*.JPG')

    # read lines csv file and create a list of lines
    lines = readCSV(filepath + 'annotation.csv')
    # setup and creation of chessboard
    chessboard =  setup(setup_image) #returns the chessboard tiles
    numOfFigures = []
    numOfWhiteColorDifferences = []
    numOfBlackColorDifferences = []
    numOfFigureDifferences = []
    numOfPreciseFigureDifferences = []
    numOfPreciseDifferences = []
    numOfDifferences = []
    if chessboard is not None:
        # Grab all images in directory
        for img in chess_images:
            for line in lines:
                if str(img.split('/')[-1]) == line[0]:
                    num_figures = int(line[1])
                    fen_positiv = line[2]

            # load and resize image for model
            img = cv2.resize(cv2.imread(img),(416,416))
            # Inference
            interference = model(img, size=416)
            # Map model output to chessboard
            chessboard = assignFigures(interference, chessboard)
            chessboard.tiles = np.rot90(chessboard.tiles, k=1, axes=(0, 1)) # axes=(1,0) ==> 90 deg clockwise; axes(0,1) ==> 90 deg counterclockwise
            # Draw matrix graphically
            #draw(chessboard)
            fen_board = board_to_fen(chessboard)
            ####### Check correct classification rate

            # number of figures
            numOfFigures.append(num_figures)

            # compare color of figures
            numOfWhiteColorDifferences.append(compare_color_fen(fen_positiv, fen_board, 'white'))
            numOfBlackColorDifferences.append(compare_color_fen(fen_positiv, fen_board, 'black'))

            # number of figures that are correct
            numOfFigureDifferences.append(compare_figureNumber_fen(fen_positiv, fen_board, False))

            # number of figures that are correct and correct type
            # also predicted figures that are not correct will be counted
            numOfPreciseFigureDifferences.append(compare_figureNumber_fen(fen_positiv, fen_board))

            # difference of figures, position, type and color
            numOfPreciseDifferences.append(compare_fen(fen_positiv, fen_board))

            # difference of figures, position and type
            numOfDifferences.append(compare_fen(fen_positiv, fen_board, False))


            #print('Number of figures: ' + str(num_figures) + ' | Difference figures: ' + str(compare_figureNumber_fen(fen_positiv,fen_board, False)) +' | Figures with correct type not found: ' + str(compare_figureNumber_fen(fen_positiv,fen_board)) +' | Precise differences: ' + str(compare_fen(fen_positiv, fen_board)) + '| Differences: ' + str(compare_fen(fen_positiv, fen_board, False)))
            # rotate back to original orientation
            chessboard.tiles = np.rot90(chessboard.tiles, k=1, axes=(1, 0)) # axes=(1,0) ==> 90 deg clockwise; axes(0,1) ==> 90 deg counterclockwise
            clear(chessboard)

        # plot the results
        order = np.argsort(numOfFigures)
        xs = np.array(numOfFigures)[order]
        y1 = np.array(numOfFigureDifferences)[order]
        y2 = np.array(numOfPreciseFigureDifferences)[order]
        y3 = np.array(numOfPreciseDifferences)[order]
        y4 = np.array(numOfDifferences)[order]
        y5 = np.array(numOfWhiteColorDifferences)[order]
        y6 = np.array(numOfBlackColorDifferences)[order]

        fig, ax = plt.subplots()
        ax.plot(xs, y1, 'g-', label='Anzahl Figuren korrekt')
        ax.plot(xs, y2, 'c-', label='Typ und Farbe korrekt')
        ax.plot(xs, y3, 'r-', label='Typ, Position und Farbe korrekt')
        ax.plot(xs, y4, 'b-', label='Typ und Position korrekt')

        ax.set_title(f)
        ax.set_xlabel('Anzahl an Figuren')
        ax.set_ylabel('Anzahl der Unterschiede')
        ax.set(xlim=(0, 40), ylim=(0, 35))
        ax.legend()
        plt.savefig('plotsAndGraphs/' + f + '_figures_check_Accuracy.png' ,dpi=300)

Using cache found in /Users/seebasti/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-2-13 torch 1.10.1 CPU

Fusing layers... 
Model Summary: 280 layers, 3108508 parameters, 0 gradients, 4.3 GFLOPs
Adding AutoShape... 
/Users/seebasti/Projects/ChessCamera/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/seebasti/Projects/ChessCamera/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/seebasti/Projects/ChessCamera/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provid